# Manifold GP Semi-Supervised Learning via Precision Matrix on 2D Manifold

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
# from xvfbwrapper import Xvfb
# vdisplay = Xvfb(width=1920, height=1080)
# vdisplay.start()

In [2]:
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
from mayavi import mlab
from importlib.resources import files
from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from manifold_gp.utils.generate_truth import groundtruth_from_mesh

Select the device: GPU or CPU.

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Load Dataset and Generate Ground Truth

In [4]:
data_path = files('manifold_gp.data').joinpath('dragon10k.stl')
nodes, faces, truth = groundtruth_from_mesh(data_path)
nodes = nodes[:1000,:]
truth = truth[:1000]
sampled_x = torch.from_numpy(nodes).float().to(device)
sampled_y = torch.from_numpy(truth).float().to(device)
(m, n) = sampled_x.shape

## Apply Noise to the Dataset

In order to simulate real case scenario we can apply a normally distributed noise to our dataset. Here we can consider to apply the noise either to the sampled data points from the manifold or to the ground truth relative to each labeled points.

### Apply noise to the manifold
Setting the variable `manifold_noise` you can apply Gaussian noise to the sampled points from the manifold.

In [5]:
manifold_noise = 0.00
noisy_x = sampled_x + manifold_noise * torch.randn(m, n).to(device)

### Apply noise to the ground truth
Setting the variable `function_noise` you can apply Gaussian noise to the training points.

In [6]:
function_noise = 0.000
noisy_y = sampled_y + function_noise * torch.randn(m).to(device)

## Training Dataset

Create the training dataset. You can set the variable `num_train` to define the number of points within your dataset.

In [7]:
num_train = 100
train_idx = torch.randperm(m)[:num_train]
train_x = noisy_x[train_idx, :]
train_y = noisy_y[train_idx]

## Test Dataset

Create the training dataset. You can set the variable `num_test` to define the number of points within your dataset.

In [8]:
num_test = 100
test_idx = torch.randperm(m)[:num_test]
test_x = noisy_x[test_idx, :]
test_y = noisy_y[test_idx]

## Initialize Kernel

Create the Riemann Matern kernel. `nu` control the kernel smoothness like in the stadard Matern kernel. `neighbors` set the number of nearest neighbors in the KNN-based discrete approximation of the Laplace-Beltrami operator and construction of differentiable eigenfunctions. `modes` set the number of the eigenfunctions used to approximate the Riemann Matern kernel. As for classical *GPyTorch* kernels the Riemann Matern kernel can be decorated using with the *ScaleKernel* to have the signal variance parameter.

In [9]:
nu = 3
neighbors = 10
modes = 50
kernel = gpytorch.kernels.ScaleKernel(RiemannMaternKernel(nu=nu, nodes=noisy_x, neighbors=neighbors, modes=modes))

## Initialize Likelihood and GP Model

Define the type of likelihood and the GP model. The GP model takes as input the training set and and labels, the likelihood and the Riemann Matern Kernel.

In [10]:
likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.GreaterThan(1e-8))
model = RiemannGP(train_x, train_y, likelihood, kernel, train_idx).to(device)

## Model Hyperparameters Setting

In [11]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-5,
    'covar_module.base_kernel.epsilon': 0.5027,
    'covar_module.base_kernel.lengthscale': 0.5054,
    'covar_module.outputscale': 1.0,
}
model.initialize(**hypers)

## Train Model

Train the GP model using the fast Lanczos precision matrix optimization. `lr` set the learning rate; `iters` defines the number of iteratios; `verbose` can be set to `True` for printing the parameters each iteration.

In [12]:
lr = 1e-1
iters = 30
verbose = True
loss = model.manifold_informed_train(lr, iters, verbose)

Iteration: 0, Loss: 9073840.000, Noise Variance: 0.003, Signal Variance: 1.000, Lengthscale: 0.505, Epsilon: 0.503
Iteration: 1, Loss: 2364105.000, Noise Variance: 0.003, Signal Variance: 1.032, Lengthscale: 0.546, Epsilon: 0.464
Iteration: 2, Loss: 1322853.875, Noise Variance: 0.003, Signal Variance: 1.056, Lengthscale: 0.580, Epsilon: 0.435
Iteration: 3, Loss: 977430.188, Noise Variance: 0.003, Signal Variance: 1.077, Lengthscale: 0.608, Epsilon: 0.412
Iteration: 4, Loss: 786985.250, Noise Variance: 0.003, Signal Variance: 1.094, Lengthscale: 0.633, Epsilon: 0.393
Iteration: 5, Loss: 656380.250, Noise Variance: 0.003, Signal Variance: 1.109, Lengthscale: 0.656, Epsilon: 0.377
Iteration: 6, Loss: 558969.062, Noise Variance: 0.003, Signal Variance: 1.122, Lengthscale: 0.676, Epsilon: 0.363
Iteration: 7, Loss: 483503.469, Noise Variance: 0.003, Signal Variance: 1.134, Lengthscale: 0.694, Epsilon: 0.350
Iteration: 8, Loss: 423727.531, Noise Variance: 0.003, Signal Variance: 1.144, Length

## Model Evaluation

In [ ]:
%%capture
likelihood.eval()
model.eval()

Getting faster predictive distributions using [LOVE](https://arxiv.org/abs/1803.06058). We compute the **mean**, the **standard deviation** and one **posterior sample**. In addition we evaluate the kernel at the first point of our dataset.

In [ ]:
%%capture
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds = likelihood(model(noisy_x))
    mean = preds.mean
    std = preds.stddev
    posterior_sample = preds.sample()
    kernel_eval = kernel(sampled_x[0, :].unsqueeze(0), sampled_x).evaluate().squeeze()
    
    # Bring data to cpu
    sampled_x = sampled_x.cpu().numpy()
    sampled_y = sampled_y.cpu().numpy()
    train_x = train_x.cpu().numpy()
    test_x = test_x.cpu().numpy()
    kernel_eval = kernel_eval.cpu().numpy()
    posterior_sample = posterior_sample.cpu().numpy()
    mean = mean.cpu().numpy()
    std = std.cpu().numpy()

## Plot Results

In [ ]:
%%capture
%%bash
jupyter nbextension install --py mayavi --user

In [ ]:
%%capture
mlab.init_notebook()
v_options = {'mode': 'sphere','scale_factor': 3e-3, 'color': (0, 0, 0)}

### Ground Truth with Sample Training Points

In [ ]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=sampled_y)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
# mlab.points3d(test_x[:,0], test_x[:,1], test_x[:,2], **v_options)
# mlab.view(0.0,180.0,0.5139171204775793)
# mlab.savefig('dragon_truth_semisupervised.png')

### Mean

In [ ]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=mean)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
# mlab.view(0.0,180.0,0.5139171204775793)
# mlab.savefig('dragon_mean_semisupervised.png')

### Standard Deviation

In [ ]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=std)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
# mlab.view(0.0,180.0,0.5139171204775793)
# mlab.savefig('dragon_std_semisupervised.png')

### One Posterior Sample

In [ ]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=posterior_sample)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
# mlab.view(0.0,180.0,0.5139171204775793)
# mlab.savefig('dragon_posterior_semisupervised.png')

### Kernel Evaluation

In [ ]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=kernel_eval)
mlab.colorbar(orientation='vertical')
mlab.points3d(sampled_x[0,0], sampled_x[0,1], sampled_x[0,2], **v_options)